In [3]:
import sqlite3
import pandas as pd
from tkinter import Tk, filedialog
import matplotlib.pyplot as plt
import os

## database initiation

In [6]:
def ensure_database_initialized():
    if not os.path.exists('database.db'):
        print("[INFO] Database file not found. Creating now...")
        create_database()
    else:
        print("file not found")

## admin loader

In [9]:

def pick_file(title="Select a file"):
    Tk().withdraw()  # Hide Tkinter root window
    return filedialog.askopenfilename(title=title)

def load_data():
    conn = sqlite3.connect("database.db")
    cur = conn.cursor()

    # Asking user which file they want to upload for each table
    print("Select the file to upload for CLASSES...")
    class_file = pick_file("Select file for Classes")
    if class_file:
        df_classes = pd.read_excel(class_file) if class_file.endswith(".xlsx") else pd.read_csv(class_file)
        for _, row in df_classes.iterrows():
            cur.execute("INSERT OR IGNORE INTO classes (class_name) VALUES (?)", (row["class_name"],))

    print("Select the file to upload for STUDENTS...")
    student_file = pick_file("Select file for Students")
    if student_file:
        df_students = pd.read_excel(student_file) if student_file.endswith(".xlsx") else pd.read_csv(student_file)
        for _, row in df_students.iterrows():
            cur.execute("INSERT OR IGNORE INTO students (student_id, student_name, class_name) VALUES (?, ?, ?)",
                        (row["student_id"], row["student_name"], row["class_name"]))

    print("Select the file to upload for TEACHERS...")
    teacher_file = pick_file("Select file for Teachers")
    if teacher_file:
        df_teachers = pd.read_excel(teacher_file) if teacher_file.endswith(".xlsx") else pd.read_csv(teacher_file)
        for _, row in df_teachers.iterrows():
            cur.execute("INSERT OR IGNORE INTO teachers (teacher_id, teacher_name) VALUES (?, ?)",
                        (row["teacher_id"], row["teacher_name"]))

    print("Select the file to upload for SUBJECTS...")
    subject_file = pick_file("Select file for Subjects")
    if subject_file:
        df_subjects = pd.read_excel(subject_file) if subject_file.endswith(".xlsx") else pd.read_csv(subject_file)
        for _, row in df_subjects.iterrows():
            cur.execute("""INSERT OR IGNORE INTO subjects 
                        (subject_id, subject_name, class_name, teacher_id) 
                        VALUES (?, ?, ?, ?)""",
                        (row["subject_id"], row["subject_name"], row["class_name"], row["teacher_id"]))

    conn.commit()
    conn.close()
    print("\nData successfully loaded into the database.")


## database initialization

In [12]:
def create_tables():
    conn= sqlite3.connect("database.db")
    cur=conn.cursor()
    # classes
    cur.execute("""
        create table if not exists classes(
        class_name text primary key)
        """
               )
    # Students
    cur.execute("""
        create table if not exists students(
        student_id text primary key,
        student_name text not null,
        class_name text,
        foreign key(class_name) references classes(class_name)
        )"""
               )

    # Teachers
    cur.execute("""
    CREATE TABLE IF NOT EXISTS teachers (
        teacher_id TEXT PRIMARY KEY,
        teacher_name TEXT NOT NULL
    )
    """)

    # Subjects
    cur.execute("""
    CREATE TABLE IF NOT EXISTS subjects (
        subject_id TEXT PRIMARY KEY,
        subject_name TEXT NOT NULL,
        class_name TEXT NOT NULL,
        teacher_id TEXT NOT NULL,
        FOREIGN KEY(class_name) REFERENCES classes(class_name),
        FOREIGN KEY(teacher_id) REFERENCES teachers(teacher_id)
    )
    """)

    # Marks 
    cur.execute("""
    CREATE TABLE IF NOT EXISTS marks (
        student_id TEXT,
        subject_id TEXT,
        marks INTEGER,
        PRIMARY KEY(student_id, subject_id),
        FOREIGN KEY(student_id) REFERENCES students(student_id),
        FOREIGN KEY(subject_id) REFERENCES subjects(subject_id)
    )
    """)
    conn.commit()
    conn.close()
    print("...database initialized successfully...")

## Student portal

In [15]:
def student_portal():
    conn = sqlite3.connect('database.db')
    cur = conn.cursor()

    student_id = input("Enter your Student ID: ").strip()
    class_name = input("Enter your Class: ").strip()

    cur.execute("""
        SELECT * FROM students WHERE student_id=? AND class_name=?
    """, (student_id, class_name))
    student = cur.fetchone()

    if not student:
        print("Invalid credentials.")
        return

    while True:
        print(f"\nWelcome, {student[1]}!")
        print("1. View subject-wise marks")
        print("2. View class rank")
        print("3. Visualize marks per subject")
        print("4. Logout")
        choice = input("Enter your choice: ")

        if choice == '1':
            view_subject_wise_marks(cur, student_id)
        elif choice == '2':
            view_rank(cur, student_id, class_name)
        elif choice == '3':
            visualize_subject_scores(cur, student_id)
        elif choice == '4':
            break
        else:
            print("Invalid choice. Try again.")

    conn.close()

def visualize_subject_scores(cur, student_id):
    cur.execute("""
        SELECT subjects.subject_name, marks.marks
        FROM marks
        JOIN subjects ON marks.subject_id = subjects.subject_id
        WHERE marks.student_id = ?
    """, (student_id,))
    data = cur.fetchall()
    if not data:
        print("No marks found.")
        return

    subjects = [row[0] for row in data]
    scores = [row[1] for row in data]

    plt.figure(figsize=(8, 4))
    plt.bar(subjects, scores, color='lightgreen')
    plt.title("Subject-wise Marks")
    plt.xlabel("Subjects")
    plt.ylabel("Marks")
    plt.ylim(0, 100)
    plt.show()

def view_rank(cur, student_id, class_name):
    # Get average of this student
    cur.execute("SELECT AVG(marks) FROM marks WHERE student_id=?", (student_id,))
    my_avg = cur.fetchone()[0] or 0.0

    # Get all students in class and their averages
    cur.execute("SELECT student_id FROM students WHERE class_name=?", (class_name,))
    all_students = cur.fetchall()

    scores = []
    for sid, in all_students:
        cur.execute("SELECT AVG(marks) FROM marks WHERE student_id=?", (sid,))
        avg = cur.fetchone()[0]
        if avg is not None:
            scores.append((sid, avg))

    scores.sort(key=lambda x: x[1], reverse=True)
    rank = next((i + 1 for i, (sid, avg) in enumerate(scores) if sid == student_id), None)

    print(f"\nYour Average Marks: {my_avg:.2f}%")
    print(f"Your Rank in Class {class_name}: {rank} out of {len(scores)}")

def view_subject_wise_marks(cur, student_id):
    cur.execute("""
        SELECT subjects.subject_name, marks.marks
        FROM marks
        JOIN subjects ON marks.subject_id = subjects.subject_id
        WHERE marks.student_id = ?
    """, (student_id,))
    
    records = cur.fetchall()
    if not records:
        print("No Marks Found!")
        return

    print("\nSubject-wise Marks:\n")
    print("Subject   ---   Marks")
    for sub, marks in records:
        print(f"{sub}   {marks}")


## teacher portal

In [ ]:
def teacher_portal():
    conn=sqlite3.connect('database.db')
    cur=conn.cursor()
    print("------------------WELCOME TO TEACHERS' PORTAL-------------------")
    teacher_id=input("Enter your Teacher id:").strip()
    cur.execute(" select * from teachers where teacher_id=?", (teacher_id,))
    teacher= cur.fetchone()

    if not teacher:
        print("Invalid Credentials")
        return
    print(f"\n WELCOME {teacher[1]}!\n")
    cur.execute("select * from subjects where teacher_id=?",(teacher_id,))
    assignments=cur.fetchall()

    if not assignments:
        print("No subjects assigned to you.")
        return

    print("You are assigned the following:")
    for sub in assignments:
        print(f"subject name: {sub[1]} of Class: {sub[2]}")
    while True:
        print("\nChoose an option:")
        print("1. Upload student marks")
        print("2. View ranked list by overall percentage")
        print("3. Visualize distribution of students by percentage")
        print("4. Export performance report as PDF")
        print("5. Logout")
        choice = input("Enter choice: ")

        if choice == '1':
            upload_marks(cur, conn, teacher_id)

        elif choice == '2':
            view_rank_list(cur, teacher_id)

        elif choice == '3':
            visualize_distribution(cur, teacher_id)

        elif choice == '4':
            export_pdf_report(cur, teacher_id)

        elif choice == '5':
            break

        else:
            print("Invalid choice. Try again.")

    conn.close()

def upload_marks(cur, conn, teacher_id):
    subject_name = input("Enter subject name: ").strip()
    class_name = input("Enter class name: ").strip()

    # Validate and get subject_id
    cur.execute("""
        SELECT subject_id FROM subjects
        WHERE subject_name = ? AND class_name = ? AND teacher_id = ?
    """, (subject_name, class_name, teacher_id))
    
    result = cur.fetchone()
    if not result:
        print("Subject not found or not assigned to you.")
        return
    subject_id = result[0]

    # Get students from the class
    cur.execute("""
        SELECT student_id, student_name FROM students
        WHERE class_name = ?
    """, (class_name,))
    students = cur.fetchall()

    if not students:
        print("No students found in this class.")
        return

    for student_id, student_name in students:
        mark = input(f"Enter marks for {student_name} ({student_id}): ")
        try:
            mark = float(mark)
            if not (0 <= mark <= 100):
                raise ValueError("Out of range")
        except:
            print("Invalid mark. Skipping.")
            continue

        cur.execute("""
            INSERT OR REPLACE INTO marks (student_id, subject_id, marks)
            VALUES (?, ?, ?)
        """, (student_id, subject_id, mark))

    conn.commit()
    print("Marks uploaded successfully.")



def view_rank_list(cur, teacher_id):
    class_name = input("Enter class name: ").strip()

    cur.execute("""
        SELECT student_id, student_name FROM students
        WHERE class_name=?
    """, (class_name,))
    students = cur.fetchall()

    results = []
    for sid, name in students:
        cur.execute("SELECT AVG(marks) FROM marks WHERE student_id=?", (sid,))
        avg = cur.fetchone()[0]
        if avg is not None:
            results.append((name, sid, avg))

    ranked = sorted(results, key=lambda x: x[2], reverse=True)

    print("\nRanked List:")
    for idx, (name, sid, avg) in enumerate(ranked, start=1):
        print(f"{idx}. {name} ({sid}) - {avg:.2f}%")

def visualize_distribution(cur, teacher_id):
    class_name = input("Enter class name: ").strip()

    cur.execute("""
        SELECT student_id FROM students
        WHERE class_name=?
    """, (class_name,))
    students = cur.fetchall()

    percent_ranges = {"90-100": 0, "80-89": 0, "70-79": 0, "60-69": 0, "<60": 0}

    for sid, in students:
        cur.execute("SELECT AVG(marks) FROM marks WHERE student_id=?", (sid,))
        avg = cur.fetchone()[0]
        if avg is None:
            continue
        if avg >= 90:
            percent_ranges["90-100"] += 1
        elif avg >= 80:
            percent_ranges["80-89"] += 1
        elif avg >= 70:
            percent_ranges["70-79"] += 1
        elif avg >= 60:
            percent_ranges["60-69"] += 1
        else:
            percent_ranges["<60"] += 1

    plt.bar(percent_ranges.keys(), percent_ranges.values(), color='skyblue')
    plt.title("Student Distribution by Percentage")
    plt.xlabel("Percentage Range")
    plt.ylabel("Number of Students")
    plt.show()

## admin portal

In [21]:
##### Add one student
def add_student():
    sid = input("Enter Student ID: ")
    sname = input("Enter Student Name: ")
    cls = input("Enter Class Name: ")

    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()
    cur.execute("INSERT OR IGNORE INTO students (student_id, student_name, class_name) VALUES (?, ?, ?)",
                (sid, sname, cls))
    conn.commit()
    conn.close()
    print("Student added.")

# Add one teacher
def add_teacher():
    tid = input("Enter Teacher ID: ")
    tname = input("Enter Teacher Name: ")

    conn = sqlite3.connect('database.db')
    cur = conn.cursor()
    cur.execute("INSERT OR IGNORE INTO teachers (teacher_id, teacher_name) VALUES (?, ?)",
                (tid, tname))
    conn.commit()
    conn.close()
    print("Teacher added.")

def reset_database():
    conn = sqlite3.connect('database.db')
    cur = conn.cursor()

    cur.execute("DROP TABLE IF EXISTS marks")
    cur.execute("DROP TABLE IF EXISTS subjects")
    cur.execute("DROP TABLE IF EXISTS teachers")
    cur.execute("DROP TABLE IF EXISTS students")
    cur.execute("DROP TABLE IF EXISTS classes")

    conn.commit()
    conn.close()

    print("All tables dropped. Now recreating...")
    create_database()


def view_all_records():
    conn = sqlite3.connect('database.db')
    cur = conn.cursor()

    print("\n--- CLASSES ---")
    for row in cur.execute("SELECT * FROM classes"):
        print(row)

    print("\n--- STUDENTS ---")
    for row in cur.execute("SELECT * FROM students"):
        print(row)

    print("\n--- TEACHERS ---")
    for row in cur.execute("SELECT * FROM teachers"):
        print(row)

    print("\n--- SUBJECTS ---")
    for row in cur.execute("SELECT * FROM subjects"):
        print(row)

    print("\n--- MARKS ---")
    for row in cur.execute("SELECT * FROM marks"):
        print(row)

    conn.close()

# Admin menu
def admin_portal():
    while True:
        print("\n--- Admin Portal ---")
        print("1. Reset Database & Tables")
        print("2. Load Data from Files")
        print("3. Add New Student")
        print("4. Add New Teacher")
        print("5. view all records")
        print("6. Exit")

        choice = input("Enter choice: ")

        if choice == '1':
            reset_database()
        elif choice == '2':
            load_data()
        elif choice == '3':
            add_student()
        elif choice == '4':
            add_teacher()
        elif choice=='5':
            view_all_records()
        elif choice == '6':
            print("Exiting Admin Portal.")
            break
        else:
            print("Invalid choice. Try again.")

In [ ]:

def main_menu():
    create_tables()
    while True:
        print("\n====================================")
        print("    Student Performance Analyzer    ")
        print("====================================")
        print("Select your role:")
        print("1. Student")
        print("2. Teacher")
        print("3. Admin (for setup)")
        print("4. Exit")
        choice = input("Enter your choice (1-4): ")

        if choice == '1':
            student_portal()
        elif choice == '2':
            teacher_portal()
        elif choice == '3':
            admin_portal()
        elif choice == '4':
            print("Exiting program. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter a number from 1 to 4.")

main_menu()

...database initialized successfully...

    Student Performance Analyzer    
Select your role:
1. Student
2. Teacher
3. Admin (for setup)
4. Exit


Enter your choice (1-4):  3



--- Admin Portal ---
1. Reset Database & Tables
2. Load Data from Files
3. Add New Student
4. Add New Teacher
5. view all records
6. Exit


Enter choice:  5



--- CLASSES ---
('10A',)
('10B',)
('5',)
('7',)

--- STUDENTS ---
('10A-1', 'priyanka', '10A')
('10A-2', 'soubhik', '10A')
('10B-1', 'riya', '10B')
('5-3', 'sayan', '5')

--- TEACHERS ---
('1', 'Alok')
('2', 'pallab')
('3', 'koushik')
('4', 'aeishekha')

--- SUBJECTS ---
('1', 'bengali', '10A', '1')
('2', 'bengali', '10B', '1')
('3', 'science', '10B', '2')
('4', 'history', '10A', '2')

--- MARKS ---
('10A-1', '1', 67)
('10A-2', '1', 89)

--- Admin Portal ---
1. Reset Database & Tables
2. Load Data from Files
3. Add New Student
4. Add New Teacher
5. view all records
6. Exit


...


    Student Performance Analyzer    
Select your role:
1. Student
2. Teacher
3. Admin (for setup)
4. Exit


Enter your choice (1-4):  1
Enter your Student ID:  10A-1
Enter your Class:  10A



Welcome, priyanka!
1. View subject-wise marks
2. View class rank
3. Visualize marks per subject
4. Logout


Enter your choice:  2



Your Average Marks: 56.00%
Your Rank in Class 10A: 2 out of 2

Welcome, priyanka!
1. View subject-wise marks
2. View class rank
3. Visualize marks per subject
4. Logout
